## model
LeNet模块

In [ ]:
import torch
from torch import nn

class LeNet(nn.Module):

    def __init__(self):
        super(LeNet,self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(1,6,5),nn.Sigmoid(),nn.MaxPool2d(2,2),
            nn.Conv2d(6,16,5),nn.Sigmoid(),nn.MaxPool2d(2,2)
        )

        
        self.fc = nn.Sequential(
            nn.Linear(16*4*4,120),
            nn.Sigmoid(),
            nn.Linear(120,84),
            nn.Sigmoid(),
            nn.Linear(84,10)
        )

    def forward(self,x):
        features = self.conv(x)
        #out = self.fc(self.flatten(features))
        out = self.fc(features.view(x.shape[0], -1))
        return out

## train
调用LeNet进行训练，并存储最好的模型

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from model import LeNet
from torchvision import datasets,transforms
import os

data_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=data_transform, download=False)
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
# 加载测试数据集
test_dataset = datasets.MNIST(root='./data', train=False, transform=data_transform, download=False)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=256, shuffle=True)
print("datasets downloaded")
# train_dataset = datasets.MNIST(root='./MNIST', train=True, transform=data_transform, download=False)
# train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)

# # 加载测试数据集
# test_dataset = datasets.MNIST(root='./MNIST', train=False, transform=data_transform, download=False)
# test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=16, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = LeNet().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

# 学习率，每隔10轮变为原来的0.1
#lr_scheduler = lr_schedule.StepLR(optimizer, step_size=10, gamma=0.1)

def train(DataLoader,model,loss_fn,optimizer):
    loss,acc,n = 0.0, 0.0, 0

    for batch,(inputs,labels) in enumerate(DataLoader):
        inputs,labels = inputs.to(device),labels.to(device)
        
        outputs = model(inputs)
        cur_loss = loss_fn(outputs,labels)

        #计算准确率
        _,pred = torch.max(outputs,axis=1)
        cur_acc = torch.sum(labels==pred) / outputs.shape[0]

        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()

        loss += cur_loss.item()
        acc += cur_acc.item()
        n = n+1
    
    aver_loss = loss / n
    aver_acc = acc / n

    print('train_loss '+ str(aver_loss)) 
    print('train_acc '+ str(aver_acc))


def val(DataLoader,model,loss_fn):
    model.eval()
    loss,acc,n=0.0, 0.0, 0

    with torch.no_grad():
        for batch,(inputs,labels) in enumerate(DataLoader):
            inputs,labels = inputs.to(device),labels.to(device)
            outputs = model(inputs)
            cur_loss = loss_fn(outputs,labels)

            _,pred = torch.max(outputs,axis=1)
            cur_acc = torch.sum(labels==pred) / outputs.shape[0]

            loss += cur_loss.item()
            acc += cur_acc.item()
            n = n + 1

        return acc / n



epoch = 10
min_acc = 0

for t in range(epoch):
    print(f'epoch {t + 1}\n-------------')

    train(train_dataloader,model,loss_fn,optimizer)

    acc = val(test_dataloader,model,loss_fn)

    if acc > min_acc :
        folder = 'save_model'
        if not os.path.exists(folder):
            os.mkdir('save_model')
        
        min_acc=acc
        print("save best model")
        torch.save(model.state_dict(),'save_model/best_model.pth')


print("Done")



datasets downloaded
epoch 1
-------------
train_loss 2.004221160868381
train_acc 0.29836546994270163
save best model
epoch 2
-------------
train_loss 0.5174802250050484
train_acc 0.8689162234042553
save best model
epoch 3
-------------
train_loss 0.22834751434782719
train_acc 0.937749335106383
save best model
epoch 4
-------------
train_loss 0.15536397269114535
train_acc 0.9547595303109352
save best model
epoch 5
-------------
train_loss 0.12247392874448858
train_acc 0.9636635638297872
save best model
epoch 6
-------------
train_loss 0.10356882889854147
train_acc 0.96875
save best model
epoch 7
-------------
train_loss 0.0900824672364174
train_acc 0.9730385638297873
save best model
epoch 8
-------------
train_loss 0.08045982571041331
train_acc 0.9756538122258288
save best model
epoch 9
-------------
train_loss 0.07317514011358961
train_acc 0.9779753989361702
save best model
epoch 10
-------------
train_loss 0.06649305911615808
train_acc 0.9797429079705097
Done
